In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

path = "/content/drive/MyDrive/SkinCancerData"
print(os.listdir(path))

In [ ]:
!pip install torch torchvision timm albumentations opencv-python matplotlib

**Loading dataset**

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

data_dir = "/content/drive/MyDrive/SkinCancerData"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(data_dir, transform=transform)

len(dataset)

**Create train-test split**

In [ ]:
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16)
test_loader = DataLoader(test_ds, batch_size=16)

**Build a model (EfficientNet)**

In [ ]:
import torch
import torch.nn as nn
from torchvision.models import efficientnet_b0

device = "cuda" if torch.cuda.is_available() else "cpu"

model = efficientnet_b0(weights='IMAGENET1K_V1')
model.classifier[1] = nn.Linear(1280, 3)
model = model.to(device)

**Train the model**

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):
    model.train()
    total = 0
    correct = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    print("Epoch", epoch+1, "| Train Accuracy:", correct / total)

**Save The Model**

In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/skin_model.pth")